<h1>Nltk</h1>

O Nltk é uma biblioteca para processamento de texto natural em python. Com ela é possível realizar classificação de textos, treinar chunkers, entre outros

In [10]:
# -*- coding: utf-8 -*-

import nltk

Primeiramente, uma função bastante útil para o processamento de textos em português

In [3]:
from unicodedata import normalize

def normalize_text(text, to_lower=True):
    text = normalize('NFKD', text).encode('ASCII', 'ignore')

    final_text = text

    if to_lower:
        final_text = text.lower()

    return final_text

In [4]:
print(normalize_text('OlÁ, tudo bem? como você vai?'))
print(normalize_text('Olá, tudo bem? como você vai?', to_lower=False))

b'ola, tudo bem? como voce vai?'
b'Ola, tudo bem? como voce vai?'


Ao realizar a normalização de textos, considere excluir casos como o verbo de ligação "é" e o símbolo de parágrafo &para;

<h2>Baixando corpora e pacotes úteis</h2
<br><br>
Vamos agora baixar exemplos de textos em português para trabalhar e alguns pacotes que serão úteis

In [ ]:
#Baixar: book
#Baixar: all-corpora
#Baixar: floresta
#Baixar: machado
#Baixar: punkt
#Baixar: stopwords

#Se houverem problemas com a interface. Abrir o terminal do Jupyter e inserir os comandos:
#python -m nltk downloader book
#python -m nltk downloader all-corpora
#python -m nltk.downloader floresta
#python -m nltk.downloader machado
#python -m nltk.downloader punkt
#python -m nltk.downloader stopwords

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


<h2>Carregando textos e conhecendo funções úteis do Nltk</h2>

<h2>Segmentação de texto</h2>

<h3>Tokenização</h3>
<br>
Tokenização é o processo de dividir uma frase em palavras particulares


In [46]:
nltk.word_tokenize('meu nome é felipe navarro')

['meu', 'nome', 'é', 'felipe', 'navarro']

<h3>Segmentação em sentenças (sent tokenization)</h3>
<br>
É o processo de quebrar um texto grande em frases individuais


In [52]:
sent_tokenizer=nltk.data.load('tokenizers/punkt/portuguese.pickle')

texto = 'Meu nome é João da Silva e gosto de passear no feriado. Recentemente comprei um carro, que quebrou ontem'

sent_tokenizer.tokenize(texto)


['Meu nome é João da Silva e gosto de passear no feriado.',
 'Recentemente comprei um carro, que quebrou ontem']

In [53]:
#Para simplificar vamos criar funções

def tokens(my_str):
    return nltk.word_tokenize(my_str)

def sent_seg(my_str):
    return sent_tokenizer.tokenize(texto)

<h2>Classificação sintática</h2>

<h3>Visualizando corpus taggeado</h3>

In [21]:
from nltk.corpus import floresta
#Visualizando as palavras
print(floresta.words()[:10])

print('----------')
#Com tags
print(floresta.tagged_words()[:10])

['Um', 'revivalismo', 'refrescante', 'O', '7_e_Meio', 'é', 'um', 'ex-libris', 'de', 'a']
----------
[('Um', '>N+art'), ('revivalismo', 'H+n'), ('refrescante', 'N<+adj'), ('O', '>N+art'), ('7_e_Meio', 'H+prop'), ('é', 'P+v-fin'), ('um', '>N+art'), ('ex-libris', 'H+n'), ('de', 'H+prp'), ('a', '>N+art')]


In [33]:
def simple_tag(tag):
    try:
        return tag.split('+')[1]
    except:
        return tag

simple_tag('>N+art')

'art'

In [27]:
#Observando por sentenças
tagged_sents = floresta.tagged_sents()

for sent in tagged_sents[:10]:
    print(sent)
    print('--------')

[('Um', '>N+art'), ('revivalismo', 'H+n'), ('refrescante', 'N<+adj')]
--------
[('O', '>N+art'), ('7_e_Meio', 'H+prop'), ('é', 'P+v-fin'), ('um', '>N+art'), ('ex-libris', 'H+n'), ('de', 'H+prp'), ('a', '>N+art'), ('noite', 'H+n'), ('algarvia', 'N<+adj'), ('.', '.')]
--------
[('É', 'P+v-fin'), ('uma', 'H+num'), ('de', 'H+prp'), ('as', '>N+art'), ('mais', '>A+adv'), ('antigas', 'H+adj'), ('discotecas', 'H+n'), ('de', 'H+prp'), ('o', '>N+art'), ('Algarve', 'H+prop'), (',', ','), ('situada', 'P+v-pcp'), ('em', 'H+prp'), ('Albufeira', 'P<+prop'), (',', ','), ('que', 'SUBJ+pron-indp'), ('continua', 'AUX+v-fin'), ('a', 'PRT-AUX<+prp'), ('manter', 'MV+v-inf'), ('os', '>N+art'), ('traços', 'H+n'), ('decorativos', 'N<+adj'), ('e', 'CO+conj-c'), ('as', '>N+art'), ('clientelas', 'H+n'), ('de', 'H+prp'), ('sempre', 'P<+adv'), ('.', '.')]
--------
[('É', 'P+v-fin'), ('um_pouco', 'ADVL+adv'), ('a', '>N+art'), ('versão', 'H+n'), ('de', 'H+prp'), ('uma', '>N+art'), ('espécie', 'H+n'), ('de', 'H+prp'),

In [34]:
#Criando sentenças simplificadas
simplified_sents = [[(n, simple_tag(t)) for n, t in sent] for sent in tagged_sents]

In [35]:
simplified_sents[0]

[('Um', 'art'), ('revivalismo', 'n'), ('refrescante', 'adj')]

In [38]:
import random
random.shuffle(simplified_sents)

num_sents = len(simplified_sents)
len_train = int(num_sents*0.8)

train = simplified_sents[:len_train]
test = simplified_sents[len_train:]

<h3>Treinando POS taggers</h3>

In [43]:
from nltk.metrics import accuracy

tagger0 = nltk.DefaultTagger('n')

print('Acuracia default tagger', tagger0.evaluate(test))

tagger1 = nltk.UnigramTagger(train, backoff=tagger0)

print('Acuracia unigram tagger', tagger1.evaluate(test))

tagger2 = nltk.BigramTagger(train, backoff=tagger1)

print('Acuracia bigram tagger', tagger2.evaluate(test))

Acuracia default tagger 0.18786394306312967
Acuracia unigram tagger 0.8811119327109714
Acuracia bigram tagger 0.8945836029207875


<h3>Testando</h3>

In [48]:
tagger2.tag(tokens('eu sou felipe navarro'))

[('eu', 'pron-pers'), ('sou', 'v-fin'), ('felipe', 'n'), ('navarro', 'n')]

<h2>Chunkers</h2>

<h3>Visualizando corpus de Name entity recognition</h3>

<a href="https://streamhacker.com/2008/12/29/how-to-train-a-nltk-chunker/">Veja aqui</a>